# Jaguar Re-ID: Exploratory Data Analysis

**Competition**: [Jaguar Re-ID](https://www.kaggle.com/competitions/jaguar-re-id)  
**Task**: Predict pairwise similarity scores for 137,270 test image pairs  
**Metric**: Identity-balanced Mean Average Precision (mAP)  

This notebook covers:
1. Dataset overview and class distribution
2. Sample image gallery per identity
3. Image property analysis (resolution, aspect ratio)
4. Near-duplicate detection via perceptual hashing
5. Test pair structure and positive/negative ratio
6. Color and texture characteristics per identity
7. Evaluation metric walkthrough

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from pathlib import Path
from PIL import Image
import imagehash
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Paths
ROOT = Path('../')
TRAIN_DIR = ROOT / 'train' / 'train'
TEST_DIR  = ROOT / 'test'  / 'test'
FIGURES   = ROOT / 'figures'
FIGURES.mkdir(exist_ok=True)

# Plotting style
plt.rcParams.update({
    'figure.dpi': 120,
    'font.family': 'DejaVu Sans',
    'axes.spines.top': False,
    'axes.spines.right': False,
})
PALETTE = sns.color_palette('tab20', 31)

print('Setup complete.')
print(f'Train dir: {TRAIN_DIR} — exists: {TRAIN_DIR.exists()}')
print(f'Test dir:  {TEST_DIR}  — exists: {TEST_DIR.exists()}')

## 1. Dataset Overview

In [ ]:
train_df = pd.read_csv(ROOT / 'train.csv')
test_df  = pd.read_csv(ROOT / 'test.csv')

print('=== TRAIN ===' )
print(f'  Images     : {len(train_df):,}')
print(f'  Identities : {train_df.ground_truth.nunique()}')
print()
print('=== TEST ===' )
print(f'  Pairs      : {len(test_df):,}')
test_images = pd.unique(test_df[['query_image', 'gallery_image']].values.ravel())
print(f'  Unique imgs: {len(test_images)}')
print()

# Count per identity
identity_counts = train_df['ground_truth'].value_counts()
print('=== IDENTITY DISTRIBUTION ===')
print(identity_counts.to_string())

## 2. Class Distribution

The dataset has significant class imbalance — from 13 images (Bernard, Ipepo) to 183 images (Marcela). The identity-balanced mAP metric compensates for this by giving each jaguar equal weight, so **rare jaguars matter just as much as common ones**.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5))

colors = [PALETTE[i] for i in range(len(identity_counts))]
bars = ax.bar(identity_counts.index, identity_counts.values, color=colors, edgecolor='white', linewidth=0.5)

# Annotate count on each bar
for bar, val in zip(bars, identity_counts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1.5,
            str(val), ha='center', va='bottom', fontsize=7, fontweight='bold')

# Reference lines
mean_count = identity_counts.mean()
ax.axhline(mean_count, color='crimson', linestyle='--', linewidth=1.2, label=f'Mean: {mean_count:.0f}')

ax.set_xlabel('Jaguar Identity', fontsize=11)
ax.set_ylabel('Number of Training Images', fontsize=11)
ax.set_title('Training Set: Images per Jaguar Identity', fontsize=13, fontweight='bold')
ax.tick_params(axis='x', rotation=45)
ax.legend(fontsize=10)
plt.tight_layout()
plt.savefig(FIGURES / 'eda_01_class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'Imbalance ratio (max/min): {identity_counts.max() / identity_counts.min():.1f}x')
print(f'Jaguars with < 20 images: {(identity_counts < 20).sum()}')
print(f'Jaguars with > 100 images: {(identity_counts > 100).sum()}')

## 3. Sample Image Gallery

Let's look at representative images from each jaguar to understand visual variability: spot patterns, viewpoints, lighting conditions.

In [ ]:
def load_img(path, size=(128, 128)):
    img = Image.open(path).convert('RGB')
    img.thumbnail(size, Image.LANCZOS)
    # Paste onto white background to normalize size
    bg = Image.new('RGB', size, (255, 255, 255))
    offset = ((size[0] - img.width) // 2, (size[1] - img.height) // 2)
    bg.paste(img, offset)
    return bg

identities = identity_counts.index.tolist()
n_ids = len(identities)  # 31
n_cols = 6
n_rows = (n_ids + n_cols - 1) // n_cols  # ceil

fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 2.2, n_rows * 2.5))
fig.suptitle('One Representative Image per Jaguar Identity', fontsize=13, fontweight='bold', y=1.01)

for idx, identity in enumerate(identities):
    row, col = divmod(idx, n_cols)
    ax = axes[row, col]
    
    # Pick the first image for this identity
    sample_file = train_df[train_df.ground_truth == identity]['filename'].iloc[0]
    img = load_img(TRAIN_DIR / sample_file)
    
    ax.imshow(img)
    n = identity_counts[identity]
    ax.set_title(f'{identity}\n({n} imgs)', fontsize=8, fontweight='bold')
    ax.axis('off')

# Hide unused axes
for idx in range(n_ids, n_rows * n_cols):
    row, col = divmod(idx, n_cols)
    axes[row, col].axis('off')

plt.tight_layout()
plt.savefig(FIGURES / 'eda_02_identity_gallery.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Multi-Image Grid per Identity

Now let's look at **multiple images per jaguar** to understand intra-identity variability — the core challenge of re-ID. We show 4 images per jaguar (where available) for the 10 most-represented individuals.

In [ ]:
top10 = identity_counts.head(10).index.tolist()
n_show = 5  # images per identity

fig, axes = plt.subplots(len(top10), n_show, figsize=(n_show * 2.2, len(top10) * 2.3))
fig.suptitle('Intra-Identity Variability: 5 Images per Jaguar (Top 10 by Count)', 
             fontsize=12, fontweight='bold', y=1.005)

for row, identity in enumerate(top10):
    files = train_df[train_df.ground_truth == identity]['filename'].tolist()
    # Sample spread across the full list to capture diverse shots
    indices = np.linspace(0, len(files)-1, n_show, dtype=int)
    sampled = [files[i] for i in indices]
    
    for col, fname in enumerate(sampled):
        ax = axes[row, col]
        img = load_img(TRAIN_DIR / fname)
        ax.imshow(img)
        ax.axis('off')
        if col == 0:
            ax.set_ylabel(identity, fontsize=9, fontweight='bold', rotation=0, 
                          labelpad=50, va='center')

plt.tight_layout()
plt.savefig(FIGURES / 'eda_03_intra_identity_variability.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Image Property Analysis

Check resolution distribution and aspect ratios — important for choosing input size and understanding if resizing will cause significant distortion.

In [ ]:
print('Scanning image properties (this may take ~30 seconds)...')

records = []
for fname in train_df['filename']:
    path = TRAIN_DIR / fname
    with Image.open(path) as img:
        w, h = img.size
        mode = img.mode
    records.append({'filename': fname, 'width': w, 'height': h, 'mode': mode,
                    'aspect': w / h, 'mpx': w * h / 1e6})

props = pd.DataFrame(records)
props = props.merge(train_df, on='filename')

print(f"\nImage modes: {props['mode'].value_counts().to_dict()}")
print(f"\nResolution stats:")
print(props[['width','height','aspect','mpx']].describe().round(2))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Width distribution
axes[0].hist(props['width'], bins=40, color='steelblue', edgecolor='white', linewidth=0.5)
axes[0].set_xlabel('Width (px)')
axes[0].set_ylabel('Count')
axes[0].set_title('Image Width Distribution')

# Height distribution
axes[1].hist(props['height'], bins=40, color='coral', edgecolor='white', linewidth=0.5)
axes[1].set_xlabel('Height (px)')
axes[1].set_title('Image Height Distribution')

# Aspect ratio
axes[2].hist(props['aspect'], bins=40, color='mediumseagreen', edgecolor='white', linewidth=0.5)
axes[2].axvline(1.0, color='red', linestyle='--', linewidth=1, label='Square')
axes[2].set_xlabel('Aspect Ratio (W/H)')
axes[2].set_title('Aspect Ratio Distribution')
axes[2].legend()

plt.suptitle('Training Image Properties', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(FIGURES / 'eda_04_image_properties.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Aspect ratio range: {props['aspect'].min():.2f} – {props['aspect'].max():.2f}")
print(f"Portrait (H>W): {(props['height'] > props['width']).sum()} images")
print(f"Landscape (W>H): {(props['width'] > props['height']).sum()} images")
print(f"Square: {(props['width'] == props['height']).sum()} images")

## 6. Near-Duplicate Detection

Burst photography often produces very similar consecutive images of the same jaguar. Near-duplicates in training:
- Help the model learn robust features (slight pose variation)
- **Risk overfitting** if duplicates appear in both train and validation splits

We use **perceptual hashing** (pHash): a compact fingerprint of image content that's robust to small changes. Images with Hamming distance ≤ 8 are considered near-duplicates.

In [ ]:
print('Computing perceptual hashes (this may take ~1 minute)...')

hashes = {}
for fname in train_df['filename']:
    img = Image.open(TRAIN_DIR / fname).convert('RGB')
    hashes[fname] = imagehash.phash(img)

print(f'Computed {len(hashes)} hashes.')

# Find near-duplicates (Hamming distance <= 8)
filenames = list(hashes.keys())
near_dups = []

for i in range(len(filenames)):
    for j in range(i+1, len(filenames)):
        dist = hashes[filenames[i]] - hashes[filenames[j]]
        if dist <= 8:
            near_dups.append({
                'img1': filenames[i], 'img2': filenames[j], 'hamming': dist,
                'id1': train_df.set_index('filename').loc[filenames[i], 'ground_truth'],
                'id2': train_df.set_index('filename').loc[filenames[j], 'ground_truth'],
            })

dups_df = pd.DataFrame(near_dups)
print(f'\nNear-duplicate pairs found: {len(dups_df)}')
if len(dups_df) > 0:
    same_id = (dups_df['id1'] == dups_df['id2']).sum()
    diff_id = (dups_df['id1'] != dups_df['id2']).sum()
    print(f'  Same identity:      {same_id} pairs  ← expected (burst shots)')
    print(f'  Different identity: {diff_id} pairs  ← suspicious!')
    print(f'\nHamming distance distribution:')
    print(dups_df['hamming'].value_counts().sort_index())

In [ ]:
if len(dups_df) > 0:
    # Per-identity near-duplicate count
    dup_counts = pd.concat([
        dups_df[dups_df['id1'] == dups_df['id2']]['id1'].value_counts()
    ]).sort_values(ascending=False)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    
    # Near-dup count per identity
    if len(dup_counts) > 0:
        dup_counts.plot(kind='bar', ax=axes[0], color='slateblue', edgecolor='white')
        axes[0].set_title('Near-Duplicate Pairs per Identity', fontweight='bold')
        axes[0].set_xlabel('Jaguar Identity')
        axes[0].set_ylabel('Duplicate Pairs')
        axes[0].tick_params(axis='x', rotation=45)
    
    # Hamming distance histogram
    axes[1].hist(dups_df['hamming'], bins=range(0, 10), color='tomato', 
                 edgecolor='white', align='left')
    axes[1].set_title('Hamming Distance Distribution', fontweight='bold')
    axes[1].set_xlabel('Hamming Distance (0=identical, 8=threshold)')
    axes[1].set_ylabel('Pair Count')
    axes[1].set_xticks(range(0, 9))
    
    plt.suptitle('Near-Duplicate Analysis', fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.savefig(FIGURES / 'eda_05_near_duplicates.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Show example near-duplicate pair
    if len(dups_df) > 0:
        example = dups_df.iloc[0]
        fig, axes = plt.subplots(1, 2, figsize=(6, 3))
        axes[0].imshow(load_img(TRAIN_DIR / example['img1'], size=(256, 256)))
        axes[0].set_title(f"{example['img1']}\n({example['id1']})", fontsize=8)
        axes[0].axis('off')
        axes[1].imshow(load_img(TRAIN_DIR / example['img2'], size=(256, 256)))
        axes[1].set_title(f"{example['img2']}\n({example['id2']}) | d={example['hamming']}", fontsize=8)
        axes[1].axis('off')
        plt.suptitle('Example Near-Duplicate Pair', fontweight='bold')
        plt.tight_layout()
        plt.savefig(FIGURES / 'eda_05b_near_duplicate_example.png', dpi=150, bbox_inches='tight')
        plt.show()
else:
    print('No near-duplicates found at threshold=8.')

## 7. Test Pair Analysis

The test set contains all 371×370 = 137,270 pairwise comparisons. Since all 31 identities appear in both train and test, we can estimate the **positive/negative ratio** to understand class balance in the evaluation.

In [ ]:
# Test image count per identity (inferred from test filenames)
# We know the test files (test_0001 to test_0371), but not their identities.
# However, we can analyze the pair structure.

total_pairs = len(test_df)
n_test_imgs = len(test_images)

print(f'Total test pairs: {total_pairs:,}')
print(f'Unique test images: {n_test_imgs}')
print(f'Expected pairs (N*(N-1)): {n_test_imgs*(n_test_imgs-1):,}')
print(f'Symmetric pairs: {total_pairs == n_test_imgs*(n_test_imgs-1)}')

# Query frequency (each test image queries all others)
query_counts = test_df['query_image'].value_counts()
print(f'\nPairs per query image (should all be {n_test_imgs-1}):')
print(f'  Min: {query_counts.min()}, Max: {query_counts.max()}, Unique values: {query_counts.nunique()}')

In [ ]:
# Estimate positive/negative ratio
# We know training has 31 identities. If similar distribution holds in test:
# Average images per identity in test ~ 371 / 31 = 12
# Positive pairs per query ~ 11 (same identity - 1)
# Negative pairs per query ~ 370 - 11 = 359

avg_test_per_id = n_test_imgs / 31
est_positives_per_query = avg_test_per_id - 1
est_negatives_per_query = n_test_imgs - avg_test_per_id
est_pos_ratio = est_positives_per_query / (n_test_imgs - 1)

print('=== ESTIMATED TEST PAIR STATISTICS ===')
print(f'Avg images per identity in test : {avg_test_per_id:.1f}')
print(f'Est. positive pairs per query   : {est_positives_per_query:.1f}')
print(f'Est. negative pairs per query   : {est_negatives_per_query:.1f}')
print(f'Est. positive pair ratio        : {est_pos_ratio:.1%}')
print()
print('=> Roughly 3% of pairs are positives (same jaguar).')
print('=> Strong class imbalance — ranking quality matters more than threshold.')

## 8. Color Characteristics per Identity

Jaguar spot patterns have characteristic color profiles. Let's check per-identity mean color values — this will tell us:
- Whether color is a useful discriminative feature
- Whether camera/lighting introduces identity-confounded biases

In [ ]:
print('Computing per-identity color statistics...')

color_records = []
for fname, identity in zip(train_df['filename'], train_df['ground_truth']):
    img = np.array(Image.open(TRAIN_DIR / fname).convert('RGB').resize((64, 64)))
    # Only consider non-white pixels (background was removed)
    mask = ~((img[:,:,0] > 240) & (img[:,:,1] > 240) & (img[:,:,2] > 240))
    if mask.sum() > 100:
        r = img[:,:,0][mask].mean()
        g = img[:,:,1][mask].mean()
        b = img[:,:,2][mask].mean()
        brightness = (img[:,:,0][mask].astype(float)**2 + 
                      img[:,:,1][mask].astype(float)**2 + 
                      img[:,:,2][mask].astype(float)**2).mean()**0.5
        color_records.append({'filename': fname, 'identity': identity,
                               'R': r, 'G': g, 'B': b, 'brightness': brightness,
                               'fg_pixels': mask.sum()})

color_df = pd.DataFrame(color_records)
id_colors = color_df.groupby('identity')[['R','G','B','brightness']].mean()
print('Done. Mean color per identity computed.')
print(id_colors.describe().round(1))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Sorted by brightness
id_sorted = id_colors.sort_values('brightness')

x = np.arange(len(id_sorted))
w = 0.25
axes[0].bar(x - w, id_sorted['R'], w, color='tomato',      label='R', alpha=0.85)
axes[0].bar(x,     id_sorted['G'], w, color='mediumseagreen', label='G', alpha=0.85)
axes[0].bar(x + w, id_sorted['B'], w, color='steelblue',   label='B', alpha=0.85)
axes[0].set_xticks(x)
axes[0].set_xticklabels(id_sorted.index, rotation=45, ha='right', fontsize=8)
axes[0].set_ylabel('Mean pixel value (foreground only)')
axes[0].set_title('Mean RGB Values per Identity (sorted by brightness)', fontweight='bold')
axes[0].legend()

# Brightness scatter
brightness_by_id = color_df.groupby('identity')['brightness']
means = brightness_by_id.mean().sort_values()
stds  = brightness_by_id.std().reindex(means.index)

axes[1].errorbar(range(len(means)), means.values, yerr=stds.values,
                 fmt='o', color='darkorange', ecolor='gray', elinewidth=1, capsize=3,
                 markersize=6)
axes[1].set_xticks(range(len(means)))
axes[1].set_xticklabels(means.index, rotation=45, ha='right', fontsize=8)
axes[1].set_ylabel('Brightness (RMS pixel value, foreground)')
axes[1].set_title('Brightness per Identity (mean ± std)', fontweight='bold')

plt.suptitle('Per-Identity Color Characteristics', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(FIGURES / 'eda_06_color_characteristics.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'Brightness range across identities: {means.min():.1f} – {means.max():.1f}')
print(f'=> Brightness varies by {means.max()-means.min():.1f} units across identities — partially discriminative')

## 9. Foreground Coverage Analysis

Since backgrounds are removed, images are mostly jaguar + white background. Let's check how much of each image is foreground (jaguar pixels) vs background — this tells us how well the segmentation worked and whether padding/cropping strategies matter.

In [ ]:
# Compute foreground fraction per image using 64x64 resized versions
fg_records = []
for fname in train_df['filename']:
    img = np.array(Image.open(TRAIN_DIR / fname).convert('RGBA').resize((64, 64)))
    # Use alpha channel if available, else use white-pixel mask
    if img.shape[-1] == 4:
        fg_frac = (img[:,:,3] > 10).mean()
    else:
        rgb = img[:,:,:3]
        white_mask = (rgb[:,:,0] > 240) & (rgb[:,:,1] > 240) & (rgb[:,:,2] > 240)
        fg_frac = (~white_mask).mean()
    fg_records.append({'filename': fname, 'fg_frac': fg_frac})

fg_df = pd.DataFrame(fg_records).merge(train_df, on='filename')

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

axes[0].hist(fg_df['fg_frac'] * 100, bins=30, color='teal', edgecolor='white')
axes[0].set_xlabel('Foreground Coverage (%)')
axes[0].set_ylabel('Count')
axes[0].set_title('Jaguar Foreground Coverage Distribution', fontweight='bold')

# Per-identity average foreground coverage
fg_by_id = fg_df.groupby('ground_truth')['fg_frac'].mean().sort_values()
fg_by_id.plot(kind='bar', ax=axes[1], color='teal', edgecolor='white')
axes[1].set_title('Mean Foreground Coverage per Identity', fontweight='bold')
axes[1].set_xlabel('Jaguar Identity')
axes[1].set_ylabel('Mean FG Fraction')
axes[1].tick_params(axis='x', rotation=45)
axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))

plt.suptitle('Foreground Coverage Analysis', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(FIGURES / 'eda_07_foreground_coverage.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'Mean foreground coverage: {fg_df["fg_frac"].mean():.1%}')
print(f'Images with <30% foreground: {(fg_df["fg_frac"] < 0.3).sum()}')
print(f'Images with >80% foreground: {(fg_df["fg_frac"] > 0.8).sum()}')

## 10. Evaluation Metric Walkthrough

The competition uses **identity-balanced mAP**. Let's walk through exactly how it's computed with a toy example.

In [ ]:
def average_precision(y_true_sorted):
    """Compute AP from a binary array sorted by descending score.
    y_true_sorted[i] = 1 if the i-th ranked item is a positive match.
    """
    n_pos = y_true_sorted.sum()
    if n_pos == 0:
        return 0.0
    precisions = []
    n_correct = 0
    for rank, label in enumerate(y_true_sorted, 1):
        if label == 1:
            n_correct += 1
            precisions.append(n_correct / rank)
    return np.mean(precisions)

# Toy example: query=Marcela, gallery of 10 images (3 Marcela, 7 others)
# Perfect model: Marcela images ranked 1, 2, 3
# Random model:  Marcela images scattered throughout
# Bad model:     Marcela images ranked 8, 9, 10

perfect = np.array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0])
random  = np.array([0, 1, 0, 0, 1, 0, 0, 0, 1, 0])
bad     = np.array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

print(f'AP (perfect model): {average_precision(perfect):.3f}')  # expect ~1.0
print(f'AP (random model):  {average_precision(random):.3f}')   # expect ~0.3-0.5
print(f'AP (bad model):     {average_precision(bad):.3f}')      # expect ~0.1

print()
print('Identity-balanced mAP = mean of AP scores across all 31 jaguar identities')
print('Each identity contributes equally regardless of image count.')
print()
print('=> Models that fail on rare jaguars (Bernard: 13 imgs) will be heavily penalized.')
print('=> We must track per-identity AP throughout training, not just overall mAP.')

## 11. Key EDA Takeaways

Summary of findings that will guide our modelling approach:

In [ ]:
print('=' * 60)
print('KEY EDA FINDINGS')
print('=' * 60)
print()
print('1. CLASS IMBALANCE')
print(f'   Ratio max/min: {identity_counts.max()}/{identity_counts.min()} = '
      f'{identity_counts.max()/identity_counts.min():.1f}x')
print('   Identity-balanced mAP equalises this — use identity-stratified CV splits.')
print()
print('2. NEAR-DUPLICATES')
print(f'   Found {len(dups_df)} near-duplicate pairs in training.')
print('   Group-by-identity splits essential to avoid data leakage in CV.')
print()
print('3. TEST PAIR RATIO')
print(f'   ~3% of test pairs are positives (same jaguar).')
print('   Ranking is what matters — not raw similarity calibration.')
print()
print('4. IMAGE PROPERTIES')
print(f'   Variable resolutions — we need to normalise input size.')
print('   Variable aspect ratios — padding to square is safer than stretching.')
print('   Background removal is done — white padding is the background.')
print()
print('5. COLOR DISCRIMINATIVITY')
print('   Some per-identity color variation — color features add small signal.')
print('   But spot pattern geometry is the primary discriminator.')
print()
print('6. MODELLING IMPLICATIONS')
print('   - Use identity-stratified GroupKFold for CV')
print('   - Pad images to square before resizing (not stretch)')
print('   - Track per-identity AP, especially for rare jaguars')
print('   - Consider flip augmentation carefully (left/right flanks differ)')